In [1]:
%load_ext jupyter_black

In [94]:
from glob import glob
from typing import Iterable, Mapping
import xarray as xr
import pandas as pd
import re
from datetime import datetime

In [92]:
def extract_times(s: pd.Series) -> pd.Series:
    df = pd.concat([s, s.str.extract(r"(?P<valid_time>\d*)\.(?P<forecast_hour>\d*)$")], axis=1)
    valid_times = pd.to_datetime(df.valid_time, format="%Y%m%d%H")
    fcst_hours = pd.to_timedelta(df["forecast_hour"].astype(int), unit="h")
    s.index = valid_times + fcst_hours
    return s

grib_files = pd.Series(glob("/media/external/data/GLOBAL.grib2.*")).pipe(extract_times)
grib_files

2022-06-04 06:00:00    /media/external/data/GLOBAL.grib2.2022060400.0006
2022-06-04 12:00:00    /media/external/data/GLOBAL.grib2.2022060400.0012
2022-06-04 00:00:00    /media/external/data/GLOBAL.grib2.2022060400.0000
2022-06-04 18:00:00    /media/external/data/GLOBAL.grib2.2022060400.0018
dtype: object

In [151]:
import re
from datetime import datetime
import IPython
from IPython import display

variables = ["{0}_P0_L100_GLL0".format(x) for x in ("TMP", "RH", "UGRD", "VGRD", "HGT")]


def grib_concat(grib_files: Mapping[datetime, str], variables: list[str]) -> xr.Dataset:
    def generate():
        for timestamp, file in grib_files.items():
            ds: xr.Dataset = xr.open_dataset(file, engine="pynio")
            yield ds.expand_dims({"validTime": [timestamp.value]})[variables]
            # pyino prints a bunch of stupid warning
            display.clear_output()

    return xr.concat(generate(), dim="validTime")


ds = grib_concat(grib_files, variables=variables).rename(
    {
        "lv_ISBL0": "Pa",
        "lat_0": "lat",
        "lon_0": "lon",
        "TMP_P0_L100_GLL0": "TEMP",
        "RH_P0_L100_GLL0": "RH",
        "UGRD_P0_L100_GLL0": "U-WIND",
        "VGRD_P0_L100_GLL0": "V-WIND",
        "HGT_P0_L100_GLL0": "GPH",
    }
)
ds

<xarray.Dataset>
Dimensions:    (validTime: 4, Pa: 7, lat: 361, lon: 720)
Coordinates:
  * validTime  (validTime) int64 1654322400000000000 ... 1654365600000000000
  * lat        (lat) float32 90.0 89.5 89.0 88.5 ... -88.5 -89.0 -89.5 -90.0
  * lon        (lon) float32 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * Pa         (Pa) float32 2e+04 2.5e+04 5e+04 7e+04 8.5e+04 9.25e+04 1e+05
Data variables:
    TEMP       (validTime, Pa, lat, lon) float32 226.5 226.5 ... 243.0 243.0
    RH         (validTime, Pa, lat, lon) float32 2.714 2.714 ... 98.46 98.46
    U-WIND     (validTime, Pa, lat, lon) float32 4.311 4.311 ... -2.473 -2.473
    V-WIND     (validTime, Pa, lat, lon) float32 3.128 3.128 ... -4.475 -4.475
    GPH        (validTime, Pa, lat, lon) float32 1.166e+04 1.166e+04 ... 163.6

In [152]:
df = ds.to_dataframe()
df

TEMP         RH    U-WIND  \
validTime           Pa       lat   lon                                      
1654322400000000000 20000.0   90.0 0.0    226.497543   2.714223  4.310657   
                                   0.5    226.497543   2.714223  4.310657   
                                   1.0    226.497543   2.714223  4.310657   
                                   1.5    226.497543   2.714223  4.310657   
                                   2.0    226.497543   2.714223  4.310657   
...                                              ...        ...       ...   
1654365600000000000 100000.0 -90.0 357.5  242.958542  98.460106 -2.473158   
                                   358.0  242.958542  98.460106 -2.473158   
                                   358.5  242.958542  98.460106 -2.473158   
                                   359.0  242.958542  98.460106 -2.473158   
                                   359.5  242.958542  98.460106 -2.473158   

                                            V-WIND           GPH  
validTime           Pa       lat   lon                            
1654322400000000000 20000.0   90.0 0.0    3.127881  11662.580078  
                                   0.5    3.127881  11662.580078  
                                   1.0    3.127881  11662.580078  
                                   1.5    3.127881  11662.580078  
                                   2.0    3.127881  11662.580078  
...                                            ...           ...  
1654365600000000000 100000.0 -90.0 357.5 -4.475143    163.594727  
                                   358.0 -4.475143    163.594727  
                                   358.5 -4.475143    163.594727  
                                   359.0 -4.475143    163.594727  
                                   359.5 -4.475143    163.594727  

[7277760 rows x 5 columns]

In [95]:
from typing import Callable, Iterable
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

class Urls:
    def __init__(self, urls:Iterable[str]):
        self._urls = np.array(urls)
        
    def __repr__(self):
        return self._urls.__repr__()

    def download(self, save_to="/media/external/data/", wait:int=10):
        for url in self._urls[:2]:
            local_filename = save_to + url.split("/")[-1]
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(local_filename, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            time.sleep(60 * wait)
            
def read_urls(func:Callable[[],str]):
    url, path_opt = func()
    def inner():
        # read the base url
        r = requests.get(url)
        r.raise_for_status()

        soup = BeautifulSoup(r.content)
        latests_run = url + soup.find_all("a")[-1].text

        r = requests.get(latests_run)
        r.raise_for_status()
        soup = BeautifulSoup(r.content)
        all_gribs:list[str] = [latests_run + a.text for a in soup.find_all("a")[1:]]

        if path_opt is None:
            return Urls(all_gribs)
            
        r = requests.get([g for g in all_gribs if g.endswith(path_opt)][0])
        r.raise_for_status()
        alpha = BeautifulSoup(r.content).find_all("a")
        return Urls([latests_run + path_opt +  a.text  for a in alpha  if a.text.endswith(".grib2")])


    return inner

@read_urls
def galwem()->Urls:
    return "https://nomads.ncep.noaa.gov/pub/data/nccf/com/557ww/prod/", None

@read_urls
def hrrr(loc="conus")->Urls:
    """loc = conus or alasia"""
    return "https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/", loc+"/"

hrrr()#.download(wait=1)

array(['https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t00z.wrfnatf00.grib2',
       'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t00z.wrfnatf01.grib2',
       'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t00z.wrfnatf02.grib2',
       ...,
       'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t16z.wrfsubhf16.grib2',
       'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t16z.wrfsubhf17.grib2',
       'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20220604/conus/hrrr.t16z.wrfsubhf18.grib2'],
      dtype='<U103')